In [1]:
import streamlit as st
import folium
from streamlit_folium import st_folium
from geopy.geocoders import Nominatim
import osmnx as ox
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
import plotly.express as px
from folium import GeoJson, GeoJsonTooltip
import xlrd
pd.set_option('display.max_rows', None)


import matplotlib
matplotlib.use("TkAgg")   # Force an interactive backend

2025-11-05 17:54:55.380 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [2]:

geolocator = Nominatim(user_agent="Navigator")

@st.cache_data(show_spinner=True, show_time = True)
def geocode_address(address):
    geolocator = Nominatim(user_agent="Navigator")
    return geolocator.geocode(address)

@st.cache_data(show_spinner=True, show_time = True)
def get_osm_features(lat, lon, tags, dist):
    return ox.features_from_point((lat, lon), tags=tags, dist=dist)

@st.cache_data
def load_pie_index(sheet):
    df = pd.read_excel("OSM features.xls", sheet_name=sheet)
    df = df.dropna(subset=["key", "value"])
    df["key"] = df["key"].astype(str).str.strip()
    df["value"] = df["value"].astype(str).str.strip()
    return df

    
def clip_to_circle(gdf, lat, lon, radius):
    if gdf.crs is None:
        gdf = gdf.set_crs(4326)
    proj_crs = gdf.estimate_utm_crs()
    gdf_proj = gdf.to_crs(proj_crs)
    center = Point(lon, lat)
    circle = gpd.GeoSeries([center], crs=4326).to_crs(proj_crs).buffer(radius)
    return gpd.clip(gdf_proj, circle).to_crs(4326)


def melt_tags(gdf, tag_keys):
    # keep onlt keys that exist in gdf
    tag_keys = [k for k in tag_keys if k in gdf.columns]
    if not tag_keys:
        raise ValueError("None of the provided tag_keys exist in the GeoDataFrame.")

    melted = (
        gdf[tag_keys]
        .stack()
        .reset_index()
        .rename(columns={"level_2": "key", 0: "value"})
    )
    melted = melted.merge(gdf.reset_index()[["id", "geometry"]], on="id")
    melted = melted.drop(columns="element")
    melted = gpd.GeoDataFrame(melted, geometry="geometry", crs=gdf.crs)
    return melted

2025-11-05 17:54:57.154 No runtime found, using MemoryCacheStorageManager
2025-11-05 17:54:57.155 No runtime found, using MemoryCacheStorageManager
2025-11-05 17:54:57.156 No runtime found, using MemoryCacheStorageManager


In [3]:
address = "Skaldevägen 60"
POI_radius=500
location = geocode_address(address)
lat, lon = location.latitude, location.longitude
lat,lon

2025-11-05 17:54:57.163 No runtime found, using MemoryCacheStorageManager
2025-11-05 17:54:57.164 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 17:54:57.317 
  command:

    streamlit run C:\Users\anita\anaconda3\envs\environment\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-11-05 17:54:57.318 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 17:54:57.318 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 17:54:57.319 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 17:54:57.620 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 17:54:57.621 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 17:54:57.6

(59.3285363, 17.9360367)

In [24]:
m = folium.Map(location=[lat, lon], zoom_start=14)         
# Add address marker
folium.Marker([lat, lon], popup=address, icon=folium.Icon(color='red', icon='home')).add_to(m)


In [6]:
import requests
import time

In [7]:

# 1. Get the street network (nodes + edges)
G = ox.graph_from_point((lat, lon), dist=POI_radius, network_type='walk')

# 2. Extract nodes only
nodes, edges = ox.graph_to_gdfs(G)


In [8]:

# 3. Prepare node coordinates
coords = list(zip(nodes.y, nodes.x))
batch_size = 100  # OpenTopoData can only take limited locations per request
elevations = []

# 4. Query the OpenTopoData API in batches
for i in range(0, len(coords), batch_size):
    batch = coords[i:i+batch_size]
    locations = "|".join([f"{lat},{lon}" for lat, lon in batch])
    url = f"https://api.opentopodata.org/v1/srtm90m?locations={locations}"
    r = requests.get(url)
    if r.status_code == 200:
        results = r.json().get('results', [])
        elevations.extend([r.get('elevation', None) for r in results])
    else:
        elevations.extend([None]*len(batch))
    time.sleep(1)  # avoid rate limit

# 5. Add node elevations
nodes["elevation"] = elevations


In [9]:

# Replace None or NaN with median (fallback)
nodes["elevation"] = pd.to_numeric(nodes["elevation"], errors="coerce")
median_elev = nodes["elevation"].median()
nodes["elevation"].fillna(median_elev, inplace=True)

# 5. Push node elevations back to the graph
for node_id, elev in zip(nodes.index, nodes["elevation"]):
    G.nodes[node_id]["elevation"] = elev

# 6. Compute edge grades (uses node elevations)
G = ox.add_edge_grades(G, add_absolute=True)


C:\Users\anita\AppData\Local\Temp\ipykernel_24184\3453039874.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  nodes["elevation"].fillna(median_elev, inplace=True)


In [17]:
import matplotlib.cm as cm
import matplotlib.colors as mcolors

edges = ox.graph_to_gdfs(G, nodes=False)
#ox.plot_graph(G, edge_color=cm.plasma(edges.grade_abs / edges.grade_abs.max()), node_size=0)


In [11]:
#edges

In [12]:
# ateempt with Folium

In [18]:
import matplotlib.pyplot as plt
grades = edges['grade_abs'].dropna()  # remove any NaN just in case

# Plot histogram
plt.figure(figsize=(8,5))
plt.hist(grades, bins=30, color='skyblue', edgecolor='black')
plt.xlabel('Street Grade (%)')
plt.ylabel('Number of Streets')
plt.title('Distribution of Street Slopes')
plt.grid(True, alpha=0.3)
plt.show()

In [21]:

import osmnx as ox
import requests
import time
import pandas as pd
import folium
import branca.colormap as cm# 8. Create a linear color scale for grade_abs
max_grade = 0.15 #edges['grade_abs'].max()
colormap = cm.LinearColormap(["yellow","orange",'red', 'purple', 'blue'], vmin=0, vmax=max_grade)
colormap.caption = 'Street Grade (%)'

In [25]:

# 10. Add edges as polylines with color based on grade
for _, row in edges.iterrows():
    # handle LineString; for MultiLineString you could iterate over .geoms
    coords = [(y, x) for x, y in row.geometry.coords]
    color = colormap(row['grade_abs'])
    folium.PolyLine(coords, color=color, weight=3, opacity=0.8).add_to(m)

# 11. Add the color scale
colormap.add_to(m)

In [26]:
m